In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
#Importing libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import statsmodels.api as sm
import pylab as py
import warnings
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import KFold, StratifiedKFold, train_test_split
from sklearn.metrics import roc_auc_score, accuracy_score, confusion_matrix, roc_curve, precision_score, recall_score, precision_recall_curve
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
warnings.simplefilter(action='ignore', category=UserWarning)
warnings.filterwarnings('ignore')
sns.set(style = 'white')

In [ ]:
df_train = pd.read_csv("../input/insurance/train_ZoGVYWq.csv")
df_test = pd.read_csv("../input/insurance/test_66516Ee_qHJgfUk.csv")

In [ ]:

df_train.shape,df_test.shape

In [ ]:
df_train.columns, df_test.columns

In [ ]:
#We will rename some of the columns:
df_train = df_train.rename(columns={'perc_premium_paid_by_cash_credit': 'prec_cash_credit', 'age_in_days': 'age','Count_3-6_months_late':'count_3-6','Count_6-12_months_late':'count_6-12','Count_more_than_12_months_late':'count_more12','application_underwriting_score':'a_u_c'})
df_test = df_test.rename(columns={'perc_premium_paid_by_cash_credit': 'prec_cash_credit', 'age_in_days': 'age','Count_3-6_months_late':'count_3-6','Count_6-12_months_late':'count_6-12','Count_more_than_12_months_late':'count_more12','application_underwriting_score':'a_u_c'})

In [ ]:
df_train.dtypes

#Here we can see that some of the variables data type is object which means that python is not able to predict about there data
#type. Therefore we will have to assign a data type for those variables seeing their nature.
#Also many of the categorical variables has different data types. So we also assign them category data type.

In [ ]:
df_train['residence_area_type'] = df_train['residence_area_type'].astype('category')
df_train['renewal'] = df_train['renewal'].astype('category')

In [ ]:
#We know that from the data set that count related variables are categorical in nature and on the other hand have many classes. So we will use "Combine Sparse Class"
#technique to combine sparse classes and than convert these variables to category data type.
df_train['count_3-6'].value_counts(normalize=True)

In [ ]:
#We can see have all the categories from 1 to 12 are very less in number compared to category 0. Therefore we will combine all the classes from 1 to 12 and will name them 1.
fil =   (df_train['count_3-6'] == 2) |(df_train['count_3-6'] == 3) | (df_train['count_3-6'] == 4) | (df_train['count_3-6'] == 5) | (df_train['count_3-6'] == 6) | (df_train['count_3-6'] == 7) | (df_train['count_3-6'] == 8) | (df_train['count_3-6'] == 9) | (df_train['count_3-6'] == 10) | (df_train['count_3-6'] == 11) | (df_train['count_3-6'] == 12) | (df_train['count_3-6'] == 13)
df_train.loc[fil, 'count_3-6'] = 1
df_train['count_3-6'].value_counts(normalize = True)

In [ ]:
#Count_6-12
fil =  (df_train['count_6-12'] == 2) |(df_train['count_6-12'] == 3) |(df_train['count_6-12'] == 4) | (df_train['count_6-12'] == 5) | (df_train['count_6-12'] == 6) | (df_train['count_6-12'] == 7) | (df_train['count_6-12'] == 8) | (df_train['count_6-12'] == 9) | (df_train['count_6-12'] == 10) | (df_train['count_6-12'] == 11) | (df_train['count_6-12'] == 12) | (df_train['count_6-12'] == 13) | (df_train['count_6-12'] == 14) | (df_train['count_6-12'] == 15) | (df_train['count_6-12'] == 17)
df_train.loc[fil, 'count_6-12'] = 1
df_train['count_6-12'].value_counts()

In [ ]:
#Count_more12
fil =  (df_train['count_more12'] == 2) |(df_train['count_more12'] == 3) |(df_train['count_more12'] == 4) | (df_train['count_more12'] == 5) | (df_train['count_more12'] == 6) | (df_train['count_more12'] == 7) | (df_train['count_more12'] == 8) | (df_train['count_more12'] == 9) | (df_train['count_more12'] == 10) | (df_train['count_more12'] == 11) 
df_train.loc[fil, 'count_more12'] = 1
df_train['count_more12'].value_counts()

In [ ]:
# We know that the "count_3-6", "count_6-12", "count_more12" are categorical variables. So we convert it to category
df_train['count_3-6'] = df_train['count_3-6'].astype('category')
df_train['count_6-12'] = df_train['count_6-12'].astype('category')
df_train['count_more12'] = df_train['count_more12'].astype('category')

In [ ]:
df_train.dtypes

In [ ]:
df_train['sourcing_channel'].value_counts(normalize = True)

In [ ]:
fil = (df_train['sourcing_channel'] == 'E') 
df_train.loc[fil,'sourcing_channel'] = 'D'
df_train['sourcing_channel'].value_counts()

In [ ]:
#We also know that "sourcing channel" is a category datatype. 
df_train['sourcing_channel'] = df_train['sourcing_channel'].astype('category')

In [ ]:
df_train['residence_area_type'].value_counts(normalize = True)

In [ ]:
df_train['renewal'].value_counts()

In [ ]:
#We will apply the smae technoques in test data set.
df_test.dtypes

In [ ]:
#here we can see that the sourcing_channel,residence_Area_ and renewal are category type.
df_test['residence_area_type'] = df_test['residence_area_type'].astype('category')

In [ ]:
df_test.dtypes

In [ ]:
#We can see have all the categories from 1 to 12 are very less in number compared to category 0. Therefore we will combine all the classes from 1 to 12 and will name them 1.
fil =   (df_test['count_3-6'] == 2) |(df_test['count_3-6'] == 3) | (df_test['count_3-6'] == 4) | (df_test['count_3-6'] == 5) | (df_test['count_3-6'] == 6) | (df_test['count_3-6'] == 7) | (df_test['count_3-6'] == 8) | (df_test['count_3-6'] == 9) | (df_test['count_3-6'] == 10) | (df_test['count_3-6'] == 11) | (df_test['count_3-6'] == 12) 
df_test.loc[fil, 'count_3-6'] = 1
df_test['count_3-6'].value_counts(normalize = True)

In [ ]:
df_test['count_3-6'].value_counts()

In [ ]:
fil =  (df_test['count_6-12'] == 2) |(df_test['count_6-12'] == 3) |(df_test['count_6-12'] == 4) | (df_test['count_6-12'] == 5) | (df_test['count_6-12'] == 6) | (df_test['count_6-12'] == 7) | (df_test['count_6-12'] == 8) | (df_test['count_6-12'] == 9) | (df_test['count_6-12'] == 10) | (df_test['count_6-12'] == 11) | (df_test['count_6-12'] == 12) | (df_test['count_6-12'] == 13) | (df_test['count_6-12'] == 14) | (df_test['count_6-12'] == 15) | (df_test['count_6-12'] == 17)
df_test.loc[fil, 'count_6-12'] = 1
df_test['count_6-12'].value_counts()

In [ ]:
fil =  (df_test['count_more12'] == 2) |(df_test['count_more12'] == 3) |(df_test['count_more12'] == 4) | (df_test['count_more12'] == 5) | (df_test['count_more12'] == 6) | (df_test['count_more12'] == 7) | (df_test['count_more12'] == 8) | (df_test['count_more12'] == 9) | (df_test['count_more12'] == 10) | (df_test['count_more12'] == 11) 
df_test.loc[fil, 'count_more12'] = 1
df_test['count_more12'].value_counts()

In [ ]:
fil = (df_test['sourcing_channel'] == 'E') 
df_test.loc[fil,'sourcing_channel'] = 'D'
df_test['sourcing_channel'].value_counts()

In [ ]:
df_test['sourcing_channel'] = df_test['sourcing_channel'].astype('category')
df_test['count_3-6'] = df_test['count_3-6'].astype('category')
df_test['count_6-12'] = df_test['count_6-12'].astype('category')
df_test['count_more12'] = df_test['count_more12'].astype('category')

In [ ]:
#Univariate analysis
customer_details = ['id', 'age', 'Income']
premium_info = ['prec_cash_credit', 'a_u_c', 'premium', 'no_of_premiums_paid']

In [ ]:
def UVA_numeric(df_train, var_group):
  ''' 
  Univariate_Analysis_numeric
  takes a group of variables (INTEGER and FLOAT) and plot/print all the descriptives and properties along with KDE.

  Runs a loop: calculate all the descriptives of i(th) variable and plot/print it
  '''

  size = len(var_group)
  plt.figure(figsize = (7*size,3), dpi = 100)
  
  #looping for each variable
  for j,i in enumerate(var_group):
    
    # calculating descriptives of variable
    mini = df_train[i].min()
    maxi = df_train[i].max()
    ran = df_train[i].max()-df_train[i].min()
    mean = df_train[i].mean()
    median = df_train[i].median()
    st_dev = df_train[i].std()
    skew = df_train[i].skew()
    kurt = df_train[i].kurtosis()

    # calculating points of standard deviation
    points = mean-st_dev, mean+st_dev

    #Plotting the variable with every information
    plt.subplot(1,size,j+1)
    sns.kdeplot(df_train[i], shade=True)
    sns.lineplot(points, [0,0], color = 'black', label = "std_dev")
    sns.scatterplot([mini,maxi], [0,0], color = 'orange', label = "min/max")
    sns.scatterplot([mean], [0], color = 'red', label = "mean")
    sns.scatterplot([median], [0], color = 'blue', label = "median")
    plt.xlabel('{}'.format(i), fontsize = 20)
    plt.ylabel('density')
    plt.title('std_dev = {}; kurtosis = {};\nskew = {}; range = {}\nmean = {}; median = {}'.format((round(points[0],2),round(points[1],2)),
                                                                                                   round(kurt,2),

                                                                                                   round(skew,2),
                                                                                                   (round(mini,2),round(maxi,2),round(ran,2)),
                                                                                                   round(mean,2),
                                                                                                   round(median,2)))


In [ ]:
UVA_numeric(df_train,customer_details)

#From above we can infer that ID in uniformly distributed because every individual has different ID. So we can drop this variable.
#Age is almost normally distributed. It seems to have some outliers on the upper side
#Income is highly skewed as there are many individuals who have very high income.

In [ ]:
UVA_numeric(df_train,premium_info)

In [ ]:
count = ['count_3-6', 'count_6-12','count_more12']
policy = ['sourcing_channel','residence_area_type', 'renewal']

In [ ]:
def UVA_category(df1, var_group):

  '''
  Univariate_Analysis_categorical
  takes a group of variables (category) and plot/print all the value_counts and barplot.
  '''
  # setting figure_size
  size = len(var_group)
  plt.figure(figsize = (7*size,5), dpi = 100)

  # for every variable
  for j,i in enumerate(var_group):
    norm_count = df_train[i].value_counts(normalize = True)
    n_uni = df_train[i].nunique()

  #Plotting the variable with every information
    plt.subplot(1,size,j+1)
    sns.barplot(norm_count, norm_count.index , order = norm_count.index)
    plt.xlabel('fraction/percent', fontsize = 20)
    plt.ylabel('{}'.format(i), fontsize = 20)
    plt.title('n_uniques = {} \n value counts \n {};'.format(n_uni,norm_count))

In [ ]:
UVA_category(df_train, count )    

In [ ]:
UVA_category(df_train, policy)    

In [ ]:
df_train.isnull().sum()

In [ ]:
df_test.isnull().sum()

In [ ]:
def UVA_outlier(data, var_group, include_outlier = True):
  '''
  Univariate_Analysis_outlier:
  takes a group of variables (INTEGER and FLOAT) and plot/print boplot and descriptives\n
  Runs a loop: calculate all the descriptives of i(th) variable and plot/print it \n\n

  data : dataframe from which to plot from\n
  var_group : {list} type Group of Continuous variables\n
  include_outlier : {bool} whether to include outliers or not, default = True\n
  '''

  size = len(var_group)
  plt.figure(figsize = (7*size,4), dpi = 100)
  
  #looping for each variable
  for j,i in enumerate(var_group):
    
    # calculating descriptives of variable
    quant25 = data[i].quantile(0.25)
    quant75 = data[i].quantile(0.75)
    IQR = quant75 - quant25
    med = data[i].median()
    whis_low = med-(1.5*IQR)
    whis_high = med+(1.5*IQR)

    # Calculating Number of Outliers
    outlier_high = len(data[i][data[i]>whis_high])
    outlier_low = len(data[i][data[i]<whis_low])

    if include_outlier == True:
      print(include_outlier)
      #Plotting the variable with every information
      plt.subplot(1,size,j+1)
      sns.boxplot(data[i], orient="v")
      plt.ylabel('{}'.format(i))
      plt.title('With Outliers\nIQR = {}; Median = {} \n 2nd,3rd  quartile = {};\n Outlier (low/high) = {} \n'.format(
                                                                                                   round(IQR,2),
                                                                                                   round(med,2),
                                                                                                   (round(quant25,2),round(quant75,2)),
                                                                                                   (outlier_low,outlier_high)
                                                                                                   ))
      
    else:
      # replacing outliers with max/min whisker
      df_train = data[var_group][:]
      df_train[i][df_train[i]>whis_high] = whis_high+1
      df_train[i][df_train[i]<whis_low] = whis_low-1
      
      # plotting without outliers
      plt.subplot(1,size,j+1)
      sns.boxplot(df_train[i], orient="v")
      plt.ylabel('{}'.format(i))
      plt.title('Without Outliers\nIQR = {}; Median = {} \n 2nd,3rd  quartile = {};\n Outlier (low/high) = {} \n'.format(
                                                                                                   round(IQR,2),
                                                                                                   round(med,2)
                                                                                                   (round(quant25,2),round(quant75,2)),
                                                                                                   (outlier_low,outlier_high)
                                                                                                   ))  



In [ ]:
UVA_outlier(df_train,customer_details)

In [ ]:
UVA_outlier(df_train,premium_info)

In [ ]:
#Bivariate Analysis
numerical = df_train[[ 'age', 'Income','prec_cash_credit', 'a_u_c', 'premium', 'no_of_premiums_paid']]

In [ ]:
correlation = numerical.corr()
correlation

In [ ]:
plt.figure(figsize=(36,6), dpi=140)
for j,i in enumerate(['pearson','kendall','spearman']):
  plt.subplot(1,3,j+1)
  correlation = numerical.dropna().corr(method=i)
  sns.heatmap(correlation, linewidth = 2)
  plt.title(i, fontsize=18)

In [ ]:
#BIVARIATE ANALYSIS: CONTINUOUS CATEGORICAL VARIABLES

#List of Hypothesis and investigation to perform under this combination.  
#1 Do age plays a role in determining the probability of renewal of insurance or not?
#2 Do income plays a role in determining the probability of renewal of insurance or not?
#3 Do premium amount plays a role in determining the probability of renewal of insurance or not?
#4 Do no of premiums paid plays a role in determining the probability of renewal of insurance or not?
#5 Do application underwriting score plays a role in determining the probability of renewal of insurance or not?
#6 Do percentage premium paid by cash plays a role in determining the probability of renewal of insurance or not?

  
def TwoSampZ(X1, X2, sigma1, sigma2, N1, N2):
  '''
  takes mean, standard deviation, and number of observations and returns p-value calculated for 2-sampled Z-Test
  '''
  from numpy import sqrt, abs, round
  from scipy.stats import norm
  ovr_sigma = sqrt(sigma1**2/N1 + sigma2**2/N2)
  z = (X1 - X2)/ovr_sigma
  pval = 2*(1 - norm.cdf(abs(z)))
  return pval
def TwoSampT(X1, X2, sd1, sd2, n1, n2):
  '''
  takes mean, standard deviation, and number of observations and returns p-value calculated for 2-sample T-Test
  '''
  from numpy import sqrt, abs, round
  from scipy.stats import t as t_dist
  ovr_sd = sqrt(sd1**2/n1 + sd2**2/n2)
  t = (X1 - X2)/ovr_sd
  df = n1+n2-2
  pval = 2*(1 - t_dist.cdf(abs(t),df))
  return pval

def Bivariate_cont_cat(df_train, cont, cat, category):
  #creating 2 samples
  x1 = df_train[cont][df_train[cat]==category][:]
  x2 = df_train[cont][~(df_train[cat]==category)][:]
  
  #calculating descriptives
  n1, n2 = x1.shape[0], x2.shape[0]
  m1, m2 = x1.mean(), x2.mean()
  std1, std2 = x1.std(), x2.mean()
  
  #calculating p-values
  t_p_val = TwoSampT(m1, m2, std1, std2, n1, n2)
  z_p_val = TwoSampZ(m1, m2, std1, std2, n1, n2)

  #table
  table = pd.pivot_table(data=df_train, values=cont, columns=cat, aggfunc = np.mean)

  #plotting
  plt.figure(figsize = (15,6), dpi=140)
  
  #barplot
  plt.subplot(1,2,1)
  sns.barplot([str(category),'not {}'.format(category)], [m1, m2])
  plt.ylabel('mean {}'.format(cont))
  plt.xlabel(cat)
  plt.title('t-test p-value = {} \n z-test p-value = {}\n {}'.format(t_p_val,
                                                                z_p_val,
                                                                table))

  # boxplot
  plt.subplot(1,2,2)
  sns.boxplot(x=cat, y=cont, data=df_train)
  plt.title('categorical boxplot')
  

In [ ]:
Bivariate_cont_cat(df_train, 'Income','renewal', 1)

In [ ]:
Bivariate_cont_cat(df_train, 'age','renewal', 1)

In [ ]:
Bivariate_cont_cat(df_train, 'no_of_premiums_paid','renewal', 1)

In [ ]:
Bivariate_cont_cat(df_train, 'a_u_c','renewal', 1)

In [ ]:
Bivariate_cont_cat(df_train, 'premium','renewal', 1)

In [ ]:
Bivariate_cont_cat(df_train, 'prec_cash_credit','renewal', 1)

#From above analysis we can infer that all the numerical variables are significantly different for different categories. Therefore all the varibales statistically and significantly affect the probaility of default and not default.

In [ ]:
#Bivariate : Categorical-Categorical
def BVA_categorical_plot(data, tar, cat):
  '''
  take data and two categorical variables,
  calculates the chi2 significance between the two variables 
  and prints the result with countplot & CrossTab
  '''
  #isolating the variables
  data = df_train[[cat,tar]][:]

  #forming a crosstab
  table = pd.crosstab(df_train[tar],df_train[cat],)
  f_obs = np.array([table.iloc[0][:].values,
                    table.iloc[1][:].values])

  #performing chi2 test
  from scipy.stats import chi2_contingency
  chi, p, dof, expected = chi2_contingency(f_obs)
  
  #checking whether results are significant
  if p<0.05:
    sig = True
  else:
    sig = False

  #plotting grouped plot
  sns.countplot(x=cat, hue=tar, data=df_train)
  plt.title("p-value = {}\n difference significant? = {}\n".format(round(p,8),sig))

  #plotting percent stacked bar plot
  #sns.catplot(ax, kind='stacked')
  ax1 = df_train.groupby(cat)[tar].value_counts(normalize=True).unstack()
  ax1.plot(kind='bar', stacked='True',title=str(ax1))
  int_level = df_train[cat].value_counts()
  

In [ ]:
BVA_categorical_plot(df_train, 'count_3-6', 'renewal')

In [ ]:
BVA_categorical_plot(df_train, 'count_6-12', 'renewal')

In [ ]:
BVA_categorical_plot(df_train, 'count_more12', 'renewal')

In [ ]:
BVA_categorical_plot(df_train, 'sourcing_channel', 'renewal')

In [ ]:
BVA_categorical_plot(df_train,'residence_area_type', 'renewal')

In [ ]:
#Preprocessing Data
#We will apply the same preprocessing techniques in both the train and test data set.
#Missing values
pd.isnull(df_train).sum()


In [ ]:
pd.isnull(df_test).sum()

In [ ]:
#Here we can see that we have missing values in both data set in variables count_3-6, count_6-12, count_12more and a_u_c
#Because count_3-6, count_6-12 and count_12more are categorical variables. Therefore we will replace missing values by its mode.
# And for a_u_c we will use median as this continuous variable has outliers.

In [ ]:
df_train['count_3-6'].value_counts()

In [ ]:
df_train['count_3-6'] = df_train['count_3-6'].fillna(0)

In [ ]:
df_test['count_3-6'].value_counts()

In [ ]:
df_test['count_3-6'] = df_test['count_3-6'].fillna(0)

In [ ]:
df_train['count_6-12'].value_counts()

In [ ]:
df_train['count_6-12'] = df_train['count_6-12'].fillna(0)

In [ ]:
df_test['count_6-12'].value_counts()

In [ ]:
df_test['count_6-12'] = df_test['count_6-12'].fillna(0)

In [ ]:
df_train['count_more12'].value_counts()

In [ ]:
df_train['count_more12'] = df_train['count_more12'].fillna(0)

In [ ]:
df_test['count_more12'].value_counts()

In [ ]:
df_test['count_more12'] = df_test['count_more12'].fillna(0)

In [ ]:
df_train.isnull().sum()

In [ ]:
df_test.isnull().sum()

In [ ]:
df_train['a_u_c'].median()

In [ ]:
df_train['a_u_c'] = df_train['a_u_c'].fillna(99.21)

In [ ]:
df_test['a_u_c'].median()

In [ ]:
df_test['a_u_c'] = df_test['a_u_c'].fillna(99.21)

In [ ]:
df_train.isnull().sum()

In [ ]:
df_test.isnull().sum()

In [ ]:
#Outlier Treatment
##Income
Q1 = df_train['Income'].quantile(0.25)
Q3 = df_train['Income'].quantile(0.75)

IQR = df_train['Income'].quantile(0.75) - df_train['Income'].quantile(0.25)
IQR

whisker_1 = Q1 - (1.5*IQR)
whisker_2 = Q3 + (1.5*IQR)

whisker_1, whisker_2

In [ ]:
#We have seen in the box plot of income that all the outliers were above the upper whisker. So will use whisker_2 value to treat outliers
df_train['Income'] = np.where(df_train['Income'] >468210, whisker_2,df_train['Income'])

In [ ]:
#Premium
Q1 = df_train['premium'].quantile(0.25)
Q3 = df_train['premium'].quantile(0.75)

IQR = df_train['premium'].quantile(0.75) - df_train['premium'].quantile(0.25)
IQR

whisker_1 = Q1 - (1.5*IQR)
whisker_2 = Q3 + (1.5*IQR)

whisker_1, whisker_2

In [ ]:
df_train['premium'] = np.where(df_train['premium'] >26400, whisker_2,df_train['premium'])

In [ ]:
#age
Q1 = df_train['age'].quantile(0.25)
Q3 = df_train['age'].quantile(0.75)

IQR = df_train['age'].quantile(0.75) - df_train['age'].quantile(0.25)
IQR

whisker_1 = Q1 - (1.5*IQR)
whisker_2 = Q3 + (1.5*IQR)

whisker_1, whisker_2

In [ ]:
df_train['age'] = np.where(df_train['premium'] >34129, whisker_2,df_train['age'])

In [ ]:
Q1 = df_train['a_u_c'].quantile(0.25)
Q3 = df_train['a_u_c'].quantile(0.75)

IQR = df_train['a_u_c'].quantile(0.75) - df_train['a_u_c'].quantile(0.25)
IQR

whisker_1 = Q1 - (1.5*IQR)
whisker_2 = Q3 + (1.5*IQR)

whisker_1, whisker_2

In [ ]:
df_train['a_u_c'] = np.where(df_train['a_u_c'] < 97.795, whisker_2,df_train['a_u_c'])

In [ ]:
Q1 = df_train['no_of_premiums_paid'].quantile(0.25)
Q3 = df_train['no_of_premiums_paid'].quantile(0.75)

IQR = df_train['no_of_premiums_paid'].quantile(0.75) - df_train['no_of_premiums_paid'].quantile(0.25)
IQR

whisker_1 = Q1 - (1.5*IQR)
whisker_2 = Q3 + (1.5*IQR)

whisker_1, whisker_2

In [ ]:
df_train['no_of_premiums_paid'] = np.where(df_train['no_of_premiums_paid'] > 24.5, whisker_2,df_train['no_of_premiums_paid'])

In [ ]:
Q1 = df_train['prec_cash_credit'].quantile(0.25)
Q3 = df_train['prec_cash_credit'].quantile(0.75)

IQR = df_train['prec_cash_credit'].quantile(0.75) - df_train['prec_cash_credit'].quantile(0.25)
IQR

whisker_1 = Q1 - (1.5*IQR)
whisker_2 = Q3 + (1.5*IQR)

whisker_1, whisker_2

In [ ]:
df_train['prec_cash_credit'] = np.where(df_train['prec_cash_credit'] > 1.2939, whisker_2,df_train['prec_cash_credit'])

In [ ]:
#We wii use same techniques to treat outliers in the test data set.
#Outlier Treatment
##Income
Q1 = df_test['Income'].quantile(0.25)
Q3 = df_test['Income'].quantile(0.75)

IQR = df_test['Income'].quantile(0.75) - df_test['Income'].quantile(0.25)
IQR

whisker_1 = Q1 - (1.5*IQR)
whisker_2 = Q3 + (1.5*IQR)

whisker_1, whisker_2

In [ ]:
#We have seen in the box plot of income that all the outliers were above the upper whisker. So will use whisker_2 value to treat outliers
df_test['Income'] = np.where(df_test['Income'] >465453.75, whisker_2,df_test['Income'])

In [ ]:
#Premium
Q1 = df_test['premium'].quantile(0.25)
Q3 = df_test['premium'].quantile(0.75)

IQR = df_test['premium'].quantile(0.75) - df_test['premium'].quantile(0.25)
IQR

whisker_1 = Q1 - (1.5*IQR)
whisker_2 = Q3 + (1.5*IQR)

whisker_1, whisker_2

In [ ]:
df_test['premium'] = np.where(df_test['premium'] >26400, whisker_2,df_test['premium'])

In [ ]:
Q1 = df_test['age'].quantile(0.25)
Q3 = df_test['age'].quantile(0.75)

IQR = df_test['age'].quantile(0.75) - df_test['age'].quantile(0.25)
IQR

whisker_1 = Q1 - (1.5*IQR)
whisker_2 = Q3 + (1.5*IQR)

whisker_1, whisker_2

In [ ]:
df_test['age'] = np.where(df_test['premium'] >34132, whisker_2,df_test['age'])

In [ ]:
Q1 = df_test['a_u_c'].quantile(0.25)
Q3 = df_test['a_u_c'].quantile(0.75)

IQR = df_test['a_u_c'].quantile(0.75) - df_test['a_u_c'].quantile(0.25)
IQR

whisker_1 = Q1 - (1.5*IQR)
whisker_2 = Q3 + (1.5*IQR)

whisker_1, whisker_2

In [ ]:
df_test['a_u_c'] = np.where(df_test['a_u_c'] < 97.795, whisker_2,df_test['a_u_c'])

In [ ]:
Q1 = df_test['no_of_premiums_paid'].quantile(0.25)
Q3 = df_test['no_of_premiums_paid'].quantile(0.75)

IQR = df_test['no_of_premiums_paid'].quantile(0.75) - df_test['no_of_premiums_paid'].quantile(0.25)
IQR

whisker_1 = Q1 - (1.5*IQR)
whisker_2 = Q3 + (1.5*IQR)

whisker_1, whisker_2

In [ ]:
df_test['no_of_premiums_paid'] = np.where(df_test['no_of_premiums_paid'] > 24.5, whisker_2,df_test['no_of_premiums_paid'])

In [ ]:
Q1 = df_test['prec_cash_credit'].quantile(0.25)
Q3 = df_test['prec_cash_credit'].quantile(0.75)

IQR = df_test['prec_cash_credit'].quantile(0.75) - df_test['prec_cash_credit'].quantile(0.25)
IQR

whisker_1 = Q1 - (1.5*IQR)
whisker_2 = Q3 + (1.5*IQR)

whisker_1, whisker_2

In [ ]:
df_test['prec_cash_credit'] = np.where(df_test['prec_cash_credit'] > 1.299, whisker_2,df_test['prec_cash_credit'])

In [ ]:
# Introducing Dummy Variables
df_train = pd.concat([df_train,pd.get_dummies(df_train['count_3-6'],prefix = str('count_3-6'),prefix_sep='_')],axis = 1)
df_train = pd.concat([df_train,pd.get_dummies(df_train['count_6-12'],prefix = str('count_6-12'),prefix_sep='_')],axis = 1)
df_train = pd.concat([df_train,pd.get_dummies(df_train['count_more12'],prefix = str('count_more12'),prefix_sep='_')],axis = 1)
df_train = pd.concat([df_train,pd.get_dummies(df_train['sourcing_channel'],prefix = str('sourcing_channel'),prefix_sep='_')],axis = 1)
df_train = pd.concat([df_train,pd.get_dummies(df_train['residence_area_type'],prefix = str('residence_area_type'),prefix_sep='_')],axis = 1)


In [ ]:
df_train.columns

In [ ]:
df_train.drop(['count_3-6'], axis = 1, inplace  = True) 
df_train.drop(['count_6-12'], axis = 1, inplace = True)
df_train.drop(['count_more12'], axis = 1, inplace = True)
df_train.drop(['sourcing_channel'], axis = 1, inplace = True)
df_train.drop(['residence_area_type'], axis = 1, inplace = True)

In [ ]:
df_train1 = df_train.copy()

In [ ]:
df_train1.head()

In [ ]:
# Introducing Dummy Variables
df_test = pd.concat([df_test,pd.get_dummies(df_test['count_3-6'],prefix = str('count_3-6'),prefix_sep='_')],axis = 1)
df_test = pd.concat([df_test,pd.get_dummies(df_test['count_6-12'],prefix = str('count_6-12'),prefix_sep='_')],axis = 1)
df_test = pd.concat([df_test,pd.get_dummies(df_test['count_more12'],prefix = str('count_more12'),prefix_sep='_')],axis = 1)
df_test = pd.concat([df_test,pd.get_dummies(df_test['sourcing_channel'],prefix = str('sourcing_channel'),prefix_sep='_')],axis = 1)
df_test = pd.concat([df_test,pd.get_dummies(df_test['residence_area_type'],prefix = str('residence_area_type'),prefix_sep='_')],axis = 1)


In [ ]:
df_test.drop(['count_3-6'], axis = 1, inplace  = True) 
df_test.drop(['count_6-12'], axis = 1, inplace = True)
df_test.drop(['count_more12'], axis = 1, inplace = True)
df_test.drop(['sourcing_channel'], axis = 1, inplace = True)
df_test.drop(['residence_area_type'], axis = 1, inplace = True)

In [ ]:
df_train.dtypes.reset_index()
df_test.dtypes.reset_index()
df_train1.dtypes.reset_index()

In [ ]:
#Now i will split the train data set into train and validation set. I will be using train1 data set.
#We will first do standardization of all the continuous variables to bring them on the same scale.
num_cols = ['age', 'Income','prec_cash_credit', 'a_u_c', 'premium', 'no_of_premiums_paid']

In [ ]:
for i in num_cols:
    df_train1[i] = np.log(df_train1[i] + 10)

In [ ]:
df_train1.drop(['id'], axis = 1, inplace  = True) 

In [ ]:
df_train1.head()

In [ ]:
y = df_train1.renewal
x = df_train1.drop(['renewal'],axis = 1)

In [ ]:
pip install -U imbalanced-learn

In [ ]:
pip install scikit-learn==0.23.1

In [ ]:
pip install imbalanced-learn==0.7.0

In [ ]:
from imblearn.combine import SMOTETomek 

In [ ]:
smk = SMOTETomek(random_state = 42)
df_train1,y_all1 = smk.fit_resample(x,y)

In [ ]:
xtrain1, xvald1, ytrain1, yvald1 = train_test_split(df_train1,y_all1,test_size=1/3, random_state=11, stratify = y_all1)
model = LogisticRegression()
model.fit(xtrain1,ytrain1)
pred = model.predict_proba(xvald1)[:,1]

In [ ]:
from sklearn.metrics import roc_curve
fpr, tpr, _ = roc_curve(yvald1,pred) 
auc = roc_auc_score(yvald1, pred) 
plt.figure(figsize=(12,8)) 
plt.plot(fpr,tpr,label="Validation AUC-ROC="+str(auc)) 
x = np.linspace(0, 1, 1000)
plt.plot(x, x, linestyle='-')
plt.xlabel('False Positive Rate') 
plt.ylabel('True Positive Rate') 
plt.legend(loc=4) 
plt.show()

In [ ]:
pred_val = model.predict(xvald1)

label_preds = pred_val

cm = confusion_matrix(yvald1,label_preds)


def plot_confusion_matrix(cm, normalized=True, cmap='bone'):
    plt.figure(figsize=[7, 6])
    norm_cm = cm
    if normalized:
        norm_cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
        sns.heatmap(norm_cm, annot=cm, fmt='g', xticklabels=['Predicted: No','Predicted: Yes'], yticklabels=['Actual: No','Actual: Yes'], cmap=cmap)

plot_confusion_matrix(cm, ['No', 'Yes'])

In [ ]:
recall_score(yvald1,pred_val)

In [ ]:
#Reverse Feature Elimination and Backward Selection
from sklearn.feature_selection import RFE
import matplotlib.pyplot as plt

In [ ]:
#Create the RFE object and rank each feature
model = LogisticRegression()
rfe = RFE(estimator = model, n_features_to_select =1, step = 1)
rfe.fit(df_train1, y_all1)

In [ ]:
ranking_df_train1 = pd.DataFrame()
ranking_df_train1['Feature_name'] = df_train1.columns
ranking_df_train1['Rank'] = rfe.ranking_

In [ ]:
ranked = ranking_df_train1.sort_values(by = ["Rank"])

In [ ]:
ranked


In [ ]:
def cv_score(ml_model, rstate = 12, thres = 0.5, cols = df_train1.columns):
    i = 1
    cv_scores = []
    df2 = df_train1.copy()
    df2 = df_train1[cols]
    
    # 5 Fold cross validation stratified on the basis of target
    kf = StratifiedKFold(n_splits=5,random_state=rstate,shuffle=True)
    for df_train1_index,vald_index in kf.split(df_train1,y_all1):
        print('\n{} of kfold {}'.format(i,kf.n_splits))
        xtr,xvl = df2.loc[df_train1_index],df2.loc[vald_index]
        ytr,yvl = y_all1.loc[df_train1_index],y_all1.loc[vald_index]
            
        # Define model for fitting on the training set for each fold
        model = ml_model
        model.fit(xtr, ytr)
        pred_probs = model.predict_proba(xvl)
        pp = []
         
        # Use threshold to define the classes based on probability values
        for j in pred_probs[:,1]:
            if j>thres:
                pp.append(1)
            else:
                pp.append(0)
         
        # Calculate scores for each fold and print
        pred_val = pp
        roc_score = roc_auc_score(yvl,pred_probs[:,1])
        recall = recall_score(yvl,pred_val)
        precision = precision_score(yvl,pred_val)
        sufix = ""
        msg = ""
        msg += "ROC AUC Score: {}, Recall Score: {:.4f}, Precision Score: {:.4f} ".format(roc_score, recall,precision)
        print("{}".format(msg))
         
         # Save scores
        cv_scores.append(roc_score)
        i+=1
    return cv_scores


In [ ]:
ref_top10_scores = cv_score(LogisticRegression(), cols = ranked['Feature_name'][:10].values, thres = 0.14)

In [ ]:
#df
num_cols = ['age', 'Income','prec_cash_credit', 'a_u_c', 'premium', 'no_of_premiums_paid']

In [ ]:
for i in num_cols:
    df_train[i] = np.log(df_train[i] + 10)

In [ ]:
for i in num_cols:
    df_test[i] = np.log(df_test[i] + 10)

In [ ]:
df_train2 = df_train.drop(['id'],axis = 1)

In [ ]:
x = df_train2.drop(['renewal'], axis =1)
y = df_train2['renewal']

In [ ]:
smk = SMOTETomek(random_state = 32)
x_t,y_t = smk.fit_resample(x,y)

In [ ]:
model = LogisticRegression(random_state=0).fit(x_t, y_t)
model.fit(x_t,y_t)

In [ ]:
df_test1 = df_test.drop(['id'], axis = 1)

In [ ]:
pred = model.predict(df_test1)

In [ ]:
df_test1

In [ ]:
test_pid = df_test['id']
sub = pd.DataFrame({'id':test_pid,'renewal':pred})

In [ ]:
df_test = df_test.merge(sub, on='id', how='left')

In [ ]:
df_test